# Valuing European Option Using the Heston Model

說明如何使用QuantLib Python以及Heston模型對歐式選擇權進行評價，文章介紹了Heston模型的主要特點：它包含了一個隨機的波動性項。與Black-Scholes-Merton過程相比，該過程假設波動性是固定的，Heston模型則假設波動性是隨機的。


In [2]:
import QuantLib as ql

# 選擇權參數
strike_price = 110.0
payoff = ql.PlainVanillaPayoff(ql.Option.Call, strike_price)

# 選擇權資料
maturity_date = ql.Date(15, 1, 2016)
spot_price = 127.62
strike_price = 130
volatility = 0.20 # 過去一年的波動性
dividend_rate =  0.0163
option_type = ql.Option.Call

risk_free_rate = 0.001
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates(ql.UnitedStates.Settlement)

calculation_date = ql.Date(8, 5, 2015)
ql.Settings.instance().evaluationDate = calculation_date

# 建立歐式選擇權
payoff = ql.PlainVanillaPayoff(option_type, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)

# 建立Heston過程
v0 = volatility*volatility  # spot variance
kappa = 0.1
theta = v0
sigma = 0.1
rho = -0.75

spot_handle = ql.QuoteHandle(
    ql.SimpleQuote(spot_price)
)
flat_ts = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, risk_free_rate, day_count)
)
dividend_yield = ql.YieldTermStructureHandle(
    ql.FlatForward(calculation_date, dividend_rate, day_count)
)
heston_process = ql.HestonProcess(flat_ts,
                                  dividend_yield,
                                  spot_handle,
                                  v0,
                                  kappa,
                                  theta,
                                  sigma,
                                  rho)

# 使用Heston模型評價選擇權
engine = ql.AnalyticHestonEngine(ql.HestonModel(heston_process),0.01, 1000)
european_option.setPricingEngine(engine)
h_price = european_option.NPV()
print("The Heston model price is",h_price)


The Heston model price is 6.533855481449102


進行了與Black-Scholes-Merton模型的對比，並指出兩種模型評價結果的差異主要來自於Heston模型將波動性模型化為CIR過程。

In [3]:
flat_vol_ts = ql.BlackVolTermStructureHandle(
    ql.BlackConstantVol(calculation_date, calendar, volatility, day_count)
)
bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                           dividend_yield, 
                                           flat_ts, 
                                           flat_vol_ts)
european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
bs_price = european_option.NPV()
print("The Black-Scholes-Merton model price is ", bs_price)


The Black-Scholes-Merton model price is  6.749271812460607


文章提供了一個最小化的例子來展示如何使用Heston模型對歐式選擇權進行評價，並與Black-Scholes-Merton模型進行了對比以供學習。